In [1]:
import bio2bel_reactome
import itertools as itt
import pandas as pd

In [2]:
manager = bio2bel_reactome.Manager()

In [3]:
top_level_pathways = manager.get_all_top_hierarchy_pathways()

In [4]:
top_level = [
    pathway
    for pathway in top_level_pathways
    if 'HSA' in pathway.resource_id
]

In [5]:
def traverse(parent_pathway):
    
    if not parent_pathway.children:
        return []
    
    result = [
        {'Source Resource': 'reactome', 
         'Source ID': pathway.resource_id, 
         'Source Name': pathway.name, 
         'Mapping Type': 'isPartOf', 
         'Target Resource': 'reactome',
         'Target ID': parent_pathway.resource_id, 
         'Target Name': parent_pathway.name,
        }
        for pathway in parent_pathway.children
    ]
        
    for pathway in parent_pathway.children:
        result.extend(
            traverse(pathway)
        )
    
    return result
    

In [6]:
reactome_hierarchy = list(itt.chain.from_iterable([
    traverse(pathway)
    for pathway in top_level
]))

In [7]:
len(reactome_hierarchy)

2335

In [8]:
df = pd.DataFrame(reactome_hierarchy)

In [9]:
df.to_csv('reactome_hierarchy.tsv', index=False, sep='\t')